1.  更改CNN的層數，
2. 更改每一層裡的filter個數或是kernel_size。
3. 更改activation function的種類（relu, leakyrelu, selu, ...)
4. 更改optimizer (RMSProp, adam, ...)
5. 更改 learning rate
6. 你想改的任何東西，可參考keras官方文件：https://keras.io

**調整CNN神經網路結構**

1. 把數據做 normalization
2. 改變 optimizer為Adam
3. 選用不同的 loss function為categorical_crossentropy，適合做分類，若為整數分類可用sparse_categorical_crossentropy
4. 用 regularization 防止 overfitting，使用Dropout，代表每次訓練時隨機忽略一部分神經元。
    實作上其實是設某個比例, 讓有些神經元的輸出不要傳到下一層, 這樣就好像那些神經元 (暫時) 不存在

In [1]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#導入資料
from keras.datasets import mnist

In [4]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [5]:
x_train = x_train/225
x_test = x_test/225

In [7]:
#只有一個 channel，需將輸入每筆資料型式要從 (28, 28) 換成 (28, 28, 1)
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [9]:
from keras.utils import np_utils

In [10]:
#將類別向量轉換為二進位制
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras import regularizers

1.做 3 次 convolution, 每次都接 max-pooling

2.filter 大小都是 3x3, max-pooling 都用 2x2 為一小區塊

filters 數目要越來越多。做完 convolution 之後要拉平、再送入一個標準的神經網路。這個神經網路設計是這樣:

1.1 個隱藏層, 使用 200 個神經元。

In [35]:
model = Sequential()

In [36]:
#設定convolution，設filter個數 32，大小是3*3，padding = 'same'代表大小相同28*28
model.add(Conv2D(32,(3,3), padding = 'same', input_shape=(28,28,1)))
model.add(Activation('relu'))

In [37]:
model.add(Dropout(0.2))

In [38]:
#max-pooling，pool大小為2*2
model.add(MaxPooling2D(pool_size=(2,2)))

In [39]:
model.add(Conv2D(64,(3,3), padding = 'same'))
model.add(Activation('relu'))

In [40]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [41]:
model.add(Flatten())
model.add(Dense(200)) #200個神經元
model.add(Activation('relu'))

In [42]:
#輸出
model.add(Dense(10))
model.add(Activation('softmax'))

In [43]:
#loss fun採用'categorical_crossentropy'，optimizer採用'Adam'
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01),metrics=['accuracy'])

In [44]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_5 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)         

In [45]:
model.fit(x_train,y_train,batch_size=100, epochs=12)


Epoch 1/12
60000/60000 [==============================] - 65s 1ms/step - loss: 0.1330 - accuracy: 0.9597
Epoch 2/12
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0598 - accuracy: 0.9814
Epoch 3/12
60000/60000 [==============================] - 64s 1ms/step - loss: 0.0521 - accuracy: 0.9846
Epoch 4/12
60000/60000 [==============================] - 60s 1ms/step - loss: 0.0497 - accuracy: 0.9851
Epoch 5/12
60000/60000 [==============================] - 59s 986us/step - loss: 0.0483 - accuracy: 0.9863
Epoch 6/12
60000/60000 [==============================] - 59s 987us/step - loss: 0.0396 - accuracy: 0.9895
Epoch 7/12
60000/60000 [==============================] - 59s 984us/step - loss: 0.0400 - accuracy: 0.9887
Epoch 8/12
60000/60000 [==============================] - 59s 984us/step - loss: 0.0409 - accuracy: 0.9889
Epoch 9/12
60000/60000 [==============================] - 59s 990us/step - loss: 0.0432 - accuracy: 0.9893
Epoch 10/12
60000/60000 [===================

In [46]:
score = model.evaluate(x_test, y_test)

10000/10000 [==============================] - ETA:  - 2s 245us/step


In [47]:
print('loss:',score[0]) #loss率較前一個模型高
print('正確率:',score[1]) #正確率較前一個模型好

loss: 0.05710483255365132
正確率: 0.9876999855041504
